# Viterbi Algorithm

---

*   Viterbi's algorithm assigns probabilities to each item in a matrix that combines rows of parts of speech and columns of words in a sequence.
*   The idea is that with this algorithm and with these probabilities it can determine what is the most probable sequence of labels for that sequence that we are giving to the model as input.
*   The viterbi algorithm ends when we have calculated the probabilities of all the elements of a matrix.




In [1]:
# Dependencies Installation
!pip install conllu
!git clone https://github.com/UniversalDependencies/UD_Spanish-AnCora.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'UD_Spanish-AnCora'...
remote: Enumerating objects: 1054, done.
remote: Counting objects: 100% (70/70), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 1054 (delta 53), reused 70 (delta 53), pack-reused 984
Receiving objects: 100% (1054/1054), 388.11 MiB | 26.10 MiB/s, done.
Resolving deltas: 100% (746/746), done.


# Loading Pre-Trained HMM Model

In [2]:
#@title Loading HMM Model Probabilities

# Numerical Manipulation
import numpy as np
# Reading & storage Transition HMM
transitionProbdict = np.load('transitionHMM.npy', allow_pickle='TRUE').item()
# # Reading & storeage Emission HMM
emissionProbdict = np.load('emissionHMM.npy', allow_pickle='TRUE').item()

In [3]:
#@ Identifying 'upos' Grammatical Categories within Corpus

# Unique Content List 
stateSet = set([w.split('|')[1] for w in list(emissionProbdict.keys())])
stateSet

{'ADJ',
 'ADP',
 'ADV',
 'AUX',
 'CCONJ',
 'DET',
 'INTJ',
 'NOUN',
 'NUM',
 'PART',
 'PRON',
 'PROPN',
 'PUNCT',
 'SCONJ',
 'SYM',
 'VERB',
 '_'}

In [5]:
#@ Enumerate categories with numbers & assign them on Viterbi's Matrix 

# Void Dictionary
# key: Grammatical Category
# Value: Row Viterbi's Matrix Index 
tagStateDict = {}
# Perambulate Procedure
for i, state in enumerate(stateSet):
  tagStateDict[state] = i
tagStateDict

{'PUNCT': 0,
 'CCONJ': 1,
 'SCONJ': 2,
 'PRON': 3,
 '_': 4,
 'PROPN': 5,
 'ADV': 6,
 'INTJ': 7,
 'SYM': 8,
 'DET': 9,
 'ADJ': 10,
 'AUX': 11,
 'PART': 12,
 'NUM': 13,
 'ADP': 14,
 'NOUN': 15,
 'VERB': 16}

# Latent States Initial Distribution

In [9]:
#@title Calculating Distribution Initial States 

# Building Corpus Initial States
initTagStateProb = {} # \rho_i^{(0)}
from conllu import parse_incr
# Void List
wordList = []
# Getting Data
data_file = open("UD_Spanish-AnCora/es_ancora-ud-dev.conllu", "r", encoding="utf-8")
count = 0 # Counter - Measure Corpus Length
for tokenlist in parse_incr(data_file):
  count += 1
  tag = tokenlist[0]['upos']
  if tag in initTagStateProb.keys():
    initTagStateProb[tag] += 1
  else:
    initTagStateProb[tag] = 1

for key in initTagStateProb.keys():
  initTagStateProb[key] /= count

initTagStateProb

{'DET': 0.36275695284159615,
 'PROPN': 0.1124546553808948,
 'ADP': 0.15538089480048367,
 'PRON': 0.06348246674727932,
 'SCONJ': 0.02418379685610641,
 'ADV': 0.056831922611850064,
 'PUNCT': 0.08222490931076179,
 'VERB': 0.021160822249093107,
 'ADJ': 0.010882708585247884,
 'CCONJ': 0.032648125755743655,
 'NOUN': 0.02720677146311971,
 '_': 0.009068923821039904,
 'INTJ': 0.0006045949214026602,
 'AUX': 0.019347037484885126,
 'NUM': 0.01995163240628779,
 'PART': 0.0018137847642079807}

In [10]:
#@title Verifying is probabilities sum is 1 (100%)
np.array([initTagStateProb[k] for k in initTagStateProb.keys()]).sum()

1.0

# Bulding Viterbi's Algorithm






Given a words sequence $\{p_1, p_2, \dots, p_n \}$, and a set of gramatical categories given by `upos` convention, let's observe on viterbi's probabilities matrix as follow:

$$
\begin{array}{c c}
\begin{array}{c c c c}
\text{ADJ} \\
\text{ADV}\\
\text{PRON} \\
\vdots \\
{}
\end{array} 
&
\left[
\begin{array}{c c c c}
\nu_1(\text{ADJ}) & \nu_2(\text{ADJ}) & \dots  & \nu_n(\text{ADJ})\\
\nu_1(\text{ADV}) & \nu_2(\text{ADV}) & \dots  & \nu_n(\text{ADV})\\ 
\nu_1(\text{PRON}) & \nu_2(\text{PRON}) & \dots  & \nu_n(\text{PRON})\\
\vdots & \vdots & \dots & \vdots \\ \hdashline
p_1 & p_2 & \dots & p_n 
\end{array}
\right] 
\end{array}
$$

Where the probabilities of the first column (for a category $i$) are given by: 

$$
\nu_1(i) = \underbrace{\rho_i^{(0)}}_{\text{Initial Probability}} \times \underbrace{P(p_1 \vert i)}_{\text{Emission}}
$$

then, for the second column (given a category $j$) will be: 

$$
\nu_2(j) = \max_i \{ \nu_1(i) \times \underbrace{P(j \vert i)}_{\text{Transition}} \times \underbrace{P(p_2 \vert j)}_{\text{Emission}} \}
$$

thus, in general the probabilities for column $t$ will be given by: 

$$
\nu_{t}(j) = \max_i \{ \overbrace{\nu_{t-1}(i)}^{\text{Previous State}} \times \underbrace{P(j \vert i)}_{\text{Transition}} \times \underbrace{P(p_t \vert j)}_{\text{Emission}} \}
$$

### Setting & Building Algorithm

In [12]:
# Natural Language Tokenizer
import nltk
nltk.download('punkt')
# Getting Word Tokenizer
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [17]:
def ViterbiMatrix(sequence, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(sequence)
  viterbiProb = np.zeros((17, len(seq)))  # Upos has 17 categories

  # First Column Initialization
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # Next Columns Computing
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # Looking at Previous Column States 
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)
  
  return viterbiProb

matrix = ViterbiMatrix('el mundo es muy pequeño')
matrix

array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 6.87234397e-09, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 3.72802611e-05, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.02735591e-09,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [1.24617364e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [0.00000000e+00, 0.0000

In [18]:
def ViterbiTags(sequence, transitionProbdict=transitionProbdict, emissionProbdict=emissionProbdict, 
            tagStateDict=tagStateDict, initTagStateProb=initTagStateProb):
  seq = word_tokenize(sequence)
  viterbiProb = np.zeros((17, len(seq)))  # upos tiene 17 categorias

  # First Column Initialization
  for key in tagStateDict.keys():
    tag_row = tagStateDict[key]
    word_tag = seq[0].lower()+'|'+key
    if word_tag in emissionProbdict.keys():
      viterbiProb[tag_row, 0] = initTagStateProb[key]*emissionProbdict[word_tag]

  # Next Columns Computing
  for col in range(1, len(seq)):
    for key in tagStateDict.keys():
      tag_row = tagStateDict[key]
      word_tag = seq[col].lower()+'|'+key
      if word_tag in emissionProbdict.keys():
        # Looking at Previous Column States 
        possible_probs = []
        for key2 in tagStateDict.keys(): 
          tag_row2 = tagStateDict[key2]
          tag_prevtag = key+'|'+key2
          if tag_prevtag in transitionProbdict.keys():
            if viterbiProb[tag_row2, col-1]>0:
              possible_probs.append(
                  viterbiProb[tag_row2, col-1]*transitionProbdict[tag_prevtag]*emissionProbdict[word_tag])
        viterbiProb[tag_row, col] = max(possible_probs)

    # Tag's sequence Build-Up
    res = []
    for i, p in enumerate(seq):
      for tag in tagStateDict.keys():
        if tagStateDict[tag] == np.argmax(viterbiProb[:, i]):
          res.append((p, tag))
      
  return res

ViterbiTags('el mundo es muy grande')

[('el', 'DET'),
 ('mundo', 'NOUN'),
 ('es', 'AUX'),
 ('muy', 'ADV'),
 ('grande', 'ADJ')]

In [19]:
ViterbiTags('estos instrumentos han de rasgar')

[('estos', 'DET'),
 ('instrumentos', 'NOUN'),
 ('han', 'AUX'),
 ('de', 'ADP'),
 ('rasgar', 'VERB')]

# Entrenamiento directo de HMM con NLTK

* clase en python (NLTK) de HMM: https://www.nltk.org/_modules/nltk/tag/hmm.html

In [20]:
#@title English Corpus Treebank Example
import nltk
nltk.download('treebank')
from nltk.corpus import treebank
train_data = treebank.tagged_sents()[:3900]

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


In [21]:
#@title Training Data Structure
train_data

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [22]:
#@title Pre-Build HMM on NLTK
from nltk.tag import hmm
tagger = hmm.HiddenMarkovModelTrainer().train_supervised(train_data)
tagger

<HiddenMarkovModelTagger 46 states and 12385 output symbols>

In [23]:
tagger.tag("Pierre Vinken will get old".split())

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 ('will', 'MD'),
 ('get', 'VB'),
 ('old', 'JJ')]

In [24]:
#@title Training Accuracy
tagger.evaluate(treebank.tagged_sents()[:3900])

<ipython-input-24-8cef1b194538>:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  tagger.evaluate(treebank.tagged_sents()[:3900])


0.9815403947224078

In [25]:
#@title Training Accuracy Function Improvement
tagger.accuracy(treebank.tagged_sents()[:3900])

0.9815403947224078